# Experiment 03  -  LRC Transients

Date:

Name:

Partner's name:

## Place a heading here to indicate what you are working on

Enter notes here as you work through the experiment. Begin with a brief statement about what you will be working on, and why. 

## You might need another heading here

As you progress through the lab, keep adding notes, and include headings to help keep things organized. We will provide much less structure in this notebook, so it is up to you to take notes as you go along. We will be checking your work during the lab.

## Measurements of Inductance, Resistance, and Capacitance

Record the measured values of L, R, and C for the components that you are using in your circuit. To estimate the uncertainties in these values, you will need to consult the manual for your DMM. I recommend storing these values as variables, but also make sure they are visible (i.e. you can use a `print()` statement to show what you have recorded.

During the lab, you will be changing the capacitance of the variable capacitor, so you may want to make an array of these values and be able to add values to it ("append"). You may also want to store the parameters output by the fitting function for each value of the capacitance. See the optional pre-lab section for examples of how to do this.

## The Rest of the Experiment

The rest of this experiment goes rather similarly to Experiment 02 on the RC transient; the main difference is that the transient is a little more interesting. You will be capturing transients on the oscilloscope for several different values of the capacitance. The aim is that you will have enough data to study the dependence of resonant frequency on capacitance. So, below you will be recording your measured values of capacitance, and repeating the analysis of your transient data for each of your capacitance values.

For the remainder of the lab notes, remember to:

- keep writing down what you are doing, at the time you are doing it.
- don't forget to include uncertainties and units for all measured quantities
- don't forget to record the names of data files, so that it is easy to keep track of things
- make sure to show and store all necessary values

You will reuse the two pieces of code from last week, the code for packing and trimming the data, and the code for fitting. We include these again here for your convenience, with the places for editing indicated by "Modify  " notes in red.

As a reminder, you need to alter the following lines when packing the data:
- the name for your uploaded raw data file
- the zoomed in flat range you use to examine the noise and measure the standard deviation
- the number of points you want to average when packing the data (you may need to decide how to trade off more averaging against having a sufficient density of points more in this experiment than the last one)
- a new index range that you can use to trim unwanted data at the start or end of the time range, toggling the `trim` flag from `0` to `1`
- the filename you will use to save your packed and trimmed data 

You need to alter the following lines when fitting the data:
- the name of your packed data file - modify the line containing `fname = ...`
- your fitting function - modify the line `def fit_function(....)` and "return ..."
- your initial guesses for the parameters
- the parameter names in "param_names = ..."


## Dataset 1

Load your first data set, determine uncertainties (note that if you only have bit-noise between two values on a few points, you may want to select `digital` which takes the $(min-max)/\sqrt{3}$ instead of the `default` which uses the standard deviation of the selected data), determine how many points to average (pack), and trim the data to the relevant region of interest.

In [ ]:
# import the  necessary libraries and rename them
import numpy as np
import array
import pandas as pd
import matplotlib.pyplot as plt

###############################################################################
# LIST OF ALL INPUTS
###############################################################################

fname = 'RCdecayVC3.csv' # file name containing your raw data

title='My Packed MicSig Data' # give your graph a title
output_name = 'RCdecayVC3_packed.csv' # provide an output filename for the packed data

# choose the data range you want to examine
indexraw_min = 0
indexraw_max = 2000

# set y-scale to correspond to this range
yregion_min=5.2
yregion_max=4.8

# select an uncertainty type
# "default" uses the standard deviation of the selected flat region
# if standard deviation is unreasonably small and/or shows only digitization noise with single bit flips,
# set uncertainty type to "digital" to take the standard deviation of a boxcar defined by the max/min values
# or, set uncertainty type to "manual" and set a value for manualsigma
uncertainty = "default"
manualsigma = 0

npac=100 # define packing factor  npac

# trim data set range before output to .csv file
trim = 0 # default is 0 to save full dataset, set trim to 1 to trim the output data then give the range
trim_min=50 # start of trimmed data by packed index
trim_max=800 # end of trimmed data by packed index

###############################################################################
# READ IN DATA
###############################################################################

# read in data - the file is assumed to be in csv format (comma separated variables). 
#Files need to be specified with a full path OR they have to be saved in the same folder 
#as the script
data = np.loadtxt(fname, delimiter=',', comments='#',usecols=(3,4),skiprows=1)
#data = np.loadtxt(fname, delimiter=',',comments='#' )
# access the data columns and assign variables xraw and yraw
#generate  an array  xraw  which is the first  column  of  data.  Note the first column is 
#indexed as  zero.
xraw = data[:,0]
#generate  an array  yraw  which is the second  column  of  data  (index  1)
yraw = data[:,1]
#generate array containing index
indexraw=np.arange(len(xraw))

# plot data versus time and data versus index
fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(121)
ax1.scatter(xraw, yraw,marker='.')
ax1.set_xlabel('Time (sec)')
ax1.set_ylabel('Voltage (V)')
ax1.set_title('My Raw MicSig Data')
ax2 = fig.add_subplot(122)
ax2.scatter(indexraw,yraw,marker='.')
ax2.set_xlabel('Index')
ax2.set_ylabel('Voltage (V)')
plt.show()

###############################################################################
# PLOT RESTRICTED INDEX RANGE OF FLAT DATA, CALCULATE UNCERTAINTY
# plots a restricted index range where the data is flat
# and calculates the standard deviation to assess the noise
###############################################################################

# change axis limits
plt.xlim(indexraw_min,indexraw_max)
plt.ylim(yregion_min,yregion_max)

# plot the data versus index
plt.scatter(indexraw,yraw,marker='.')

# This next command displays the index plot. 
plt.show()

#calculate and display the mean and standard deviation of the data that you have zoomed in on.
y_ave = np.mean(yraw[indexraw_min:indexraw_max])
y_std = np.std(yraw[indexraw_min:indexraw_max])
print('mean = ',y_ave)
print('standard deviation = ', y_std)

# display a histogram of the data
hist,bins = np.histogram(yraw[indexraw_min:indexraw_max],bins=20)
plt.bar(bins[:-1],hist,width=bins[1]-bins[0])
plt.ylim(0,1.2*np.max(hist))
plt.xlabel('y_raw (Volts)')
plt.ylabel('Number of occurences')
plt.show()

###############################################################################
# PACK AND TRIM DATA
###############################################################################

#define a function  to pack the data
def pack(A,p):
  # A is an array, and p is the packing factor
  B = np.zeros(len(A)//p)
  i = 1
  while i-1<len(B):
    B[i-1] = np.mean(A[p*(i-1):p*i])
    i += 1
  return B
# pack the data
x=pack(xraw,npac)
y=pack(yraw,npac)

#create a vector that also has the integer index (index = 0,1,2 ... length-1)
length=len(x)
#print(length)
index = np.arange(length)

#create a vector that contains fixed uncertainty for x values (in this case set to zero
sigmax = [0]*length
#print(sigmax)

#Create a vector that contains uncertainty of averaged y values. 
#sigmayraw is your estimate of the uncertainty in individual raw data points

#Here it is taking that value from your previous statistics code 
#If the data sampled shows only a small amount of digitization noise, and the standard deviation is unreasonably small, 
#select "digital" above to take the stdev of a boxcar of the width of the digitization
#If you think the standard deviation of your data is an underestimate of the uncertainty,
#you can also enter a value by hand - just change the line that defines simayraw

if uncertainty == "digital":
    sigmayraw = 0.5 * (np.max(yraw[indexraw_min:indexraw_max])-np.min(yraw[indexraw_min:indexraw_max]))/np.sqrt(3)
    print("digital uncertainty used:",sigmayraw)
elif uncertainty == "manual":
    sigmayraw = manualsigma
    print("manually specified uncertainty used:",sigmayraw)
else:
    sigmayraw = y_std
    

#sigmaymean is the uncertainty of y after averaging npac points together
#sigmay is an array of uncertainties, all of the same value as sigmaymean
sigmaymean = sigmayraw/np.sqrt(npac)
sigmay = [sigmaymean]*length

print('packing is done by averaging', npac, 'data points')

if trim == 1:
    index_min=trim_min
    index_max=trim_max
    print('packed data has been trimmed from ', index_min, 'to ', index_max)
else:
    index_min=0
    index_max=length
    print('packed data contains full data set')

# plot the trimmed data versus index
plt.errorbar(index,y,yerr=sigmay,marker='.',linestyle='')
## marker='o' : use markers to indicate each data point (x_1,y_1),(x_2,y_2)
## linestyle= '' : no line is drawn to connect the data points
## linestyle= '-' : a line is drawn to connect the data points

# change axis limits
plt.xlim(index_min,index_max)

# add axis labels
plt.title('Packed and Trimmed Data For Storage')
plt.xlabel('Index')
plt.ylabel('Voltage (V)')
plt.show()

# Create Array and output as CSV file in four-column format with two-row headers

header = [np.array(['time','u[time]','Voltage','u[Voltage]']), 
np.array(['(sec)','(sec)','(V)','(V)'])]
d1 = [x[index_min:index_max] , sigmax[index_min:index_max] , y[index_min:index_max] , sigmay[index_min:index_max]]
d1_a = np.array(d1)
df = pd.DataFrame(np.transpose(d1_a), columns=header )   
    
# print(df)


###############################################################################
# SAVE DATA TO FILE
###############################################################################

csv_data = df.to_csv(output_name, index = False)
print('Packed Data Stored in ', output_name)

Next, fit the data.

In [ ]:
# Load python packages
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

###############################################################################
# DEFINED FITTING FUNCTIONS
###############################################################################

def sine_func(x, amplitude, freq, phase):
    return amplitude * np.sin(2.0 * np.pi * freq * x + phase)

def offset_sine_func(x, amplitude, freq, phase, offset):
    return (amplitude * np.sin(2.0 * np.pi * freq * x + phase)) + offset

def exponential_func(x, amplitude, tau, voffset):
    return amplitude * np.exp(x/(-1.0*tau)) + voffset

def ringdown_function(x, amplitude, tau, resonantf, phase):
    return amplitude * np.exp(-x/tau) * np.cos(2.0*np.pi * resonantf * x + phase)

def linear_func(x, slope, intercept):
    return slope * x + intercept

###############################################################################
# LIST OF ALL INPUTS
###############################################################################

# Name of the data file
fname = "sintest2_pack.csv"

# Names and units of data columns from fname
x_name = "Time"
x_units = "s"
y_name = "Voltage"
y_units = "V"

# Modify to change the fitting function, parameter names and to set initial parameter guesses
fit_function = sine_func
param_names = ("amplitude", "frequency", "phase")
guesses = (1.0, 900, 0.1)

# Flags for optional features
show_covariance_matrix = False
set_xy_boundaries = False
lower_x = -0.01 # these values ignored if set_xy_boundaries = False
upper_x = 0.01
lower_y = -1
upper_y = 1

###############################################################################
# LOAD DATA
###############################################################################

# load the file fname and skip the first 'skiprows' rows
data = np.loadtxt(fname, delimiter=",", comments="#", usecols=(0, 1, 2, 3), skiprows=2)

# Assign the data file columns to variables for later use
x = data[:, 0]
y = data[:, 2]
y_sigma = data[:, 3]

###############################################################################
# INITIAL PLOT OF THE DATA
###############################################################################

# Define 500 points spanning the range of the x-data; for plotting smooth curves
xtheory = np.linspace(min(x), max(x), 500)

# Compare the guessed curve to the data for visual reference
y_guess = fit_function(xtheory, *guesses)
plt.errorbar(x, y, yerr=y_sigma, marker=".", linestyle="", label="Measured data")
plt.plot(
    xtheory,
    y_guess,
    marker="",
    linestyle="-",
    linewidth=1,
    color="g",
    label="Initial parameter guesses",
)
plt.xlabel(f"{x_name} [{x_units}]")
plt.ylabel(f"{y_name} [{y_units}]")
plt.title(r"Comparison between the data and the intial parameter guesses")
plt.legend(loc="best", numpoints=1)
plt.show()

# calculate the value of the model at each of the x-values of the data set
y_fit = fit_function(x, *guesses)

# Residuals are the difference between the data and theory
residual = y - y_fit

# Plot the residuals
plt.errorbar(x, residual, yerr=y_sigma, marker=".", linestyle="", label="residuals")
plt.xlabel(f"{x_name} [{x_units}]")
plt.ylabel(f"Residual y-y_fit [{y_units}]")
plt.title("Residuals using initial parameter guesses")
plt.show()

###############################################################################
# PERFORM THE FIT AND PRINT RESULTS
###############################################################################

# Use curve_fit to perform the fit
# fit_function: defined above to choose a specific fitting function 
# fit_params: holds the resulting fit parameters
# fit_cov: the covariance matrix between all the parameters
#          (used to extract fitting parameter uncertanties)
# maxfev=10**5: maximum number of fitting procedure iterations before giving up
# absolute_sigma=True: uncertainties are treated as absolute (not relative)
fit_params, fit_cov = curve_fit(
    fit_function, x, y, sigma=y_sigma, 
    p0=guesses,absolute_sigma=True, maxfev=10**5)

# Define the function that calculates chi-squared
def chi_square(fit_parameters, x, y, sigma):
    dof = len(x) - len(fit_params)
    return np.sum((y - fit_function(x, *fit_parameters)) ** 2 / sigma**2)/dof

# Calculate and print reduced chi-squared
chi2 = chi_square(fit_params, x, y, y_sigma)
print("Chi-squared = ", chi2)

# Calculate the uncertainties in the fit parameters
fit_params_error = np.sqrt(np.diag(fit_cov))

# Print the fit parameters with uncertianties
print("\nFit parameters:")
for i in range(len(fit_params)):
    print(f"   {param_names[i]} = {fit_params[i]:.3e} ± {fit_params_error[i]:.3e}")
print("\n")

# (Optional) Print the covariance between all variables
if show_covariance_matrix:
    print("Covariance between fit parameters:")
    for i, fit_covariance in enumerate(fit_cov):
        for j in range(i+1,len(fit_covariance)):
            print(f"   {param_names[i]} and {param_names[j]}: {fit_cov[i,j]:.3e}")
    print("\n")

# residual is the difference between the data and model
x_fitfunc = np.linspace(min(x), max(x), len(x))
y_fitfunc = fit_function(x_fitfunc, *fit_params)
y_fit = fit_function(x, *fit_params)
residual = y-y_fit

###############################################################################
# PRODUCE A MULTIPANEL PLOT, WITH SCATTER PLOT, RESIDUALS AND RESIDUALS HISTOGRAM
###############################################################################

# The size of the canvas
fig = plt.figure(figsize=(7,15))

# The scatter plot
ax1 = fig.add_subplot(311)
ax1.errorbar(x,y,yerr=y_sigma,marker='.',linestyle='',label="Measured data")
ax1.plot(x_fitfunc, y_fitfunc, marker="", linestyle="-", linewidth=2,color="r", label="Fit")
ax1.set_xlabel(f"{x_name} [{x_units}]")
ax1.set_ylabel(f"{y_name} [{y_units}]")
ax1.set_title('Best Fit of Function to Data')

# (Optional) set the x and y boundaries of your plot
if set_xy_boundaries:
    plt.xlim(lower_x,upper_x)
    plt.ylim(lower_y,upper_y)
# Show the legend. loc='best' places it where the date are least obstructed
ax1.legend(loc='best',numpoints=1)

# The residuals plot
ax2 = fig.add_subplot(312)
ax2.errorbar(x, residual, yerr=y_sigma,marker='.', linestyle='', label="Residual (y-y_fit)")
ax2.hlines(0,np.min(x),np.max(x),lw=2,alpha=0.8)
ax2.set_xlabel(f"{x_name} [{x_units}]")
ax2.set_ylabel(f"y-y_fit [{y_units}]")
ax2.set_title('Residuals for the Best Fit')
ax2.legend(loc='best',numpoints=1)

# Histogram of the residuals
ax3 = fig.add_subplot(313)
hist,bins = np.histogram(residual,bins=30)
ax3.bar(bins[:-1],hist,width=bins[1]-bins[0])
ax3.set_ylim(0,1.2*np.max(hist))
ax3.set_xlabel(f"y-y_fit [{y_units}]")
ax3.set_ylabel('Number of occurences')
ax3.set_title('Histogram of the Residuals')

# Save a copy of the figure as a png 
plt.savefig('FittingResults.png')

# Show the plot
plt.show()

Consider creating arrays for each of the fit parameters: `amplitude`, `tau`, `resonantfreq`, `phase`, `voffset` that can be apppended with each additional fit.

## Dataset 2 (and so-on)

Repeat the this packing and fitting process with each data set until you have enough results to generate a plot of the fit parameters vs. capacitance. Remember to keep notes for each dataset, and make comments on the fit. If you are not satisfied with the fit quality, you may need to go back to the "pack and trim" step to prepare your data (e.g. if uncertainty is under/over-estimated or the residuals show you may have captured part of the data would not be described by the model). If you do this, make sure to comment on what you saw and why you adjusted the parameters you did.
If you don't include the full code blocks here, make sure to store the results of the packing and fitting in files with different names, and remember to record the names here.